In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## Basic App: Question & Answering from a Document

In [2]:
from langchain_openai import OpenAI

In [3]:
llm = OpenAI()

**Load the text file**

In [4]:
from langchain.document_loaders import TextLoader

In [5]:
loader = TextLoader("data/be-good-and-how-not-to-die.txt")

In [7]:
document = loader.load()

**The document is loaded as a Python list with metadata**

In [8]:
print(type(document))

<class 'list'>


In [9]:
print(len(document))

1


In [10]:
print(document[0].metadata)

{'source': 'data/be-good-and-how-not-to-die.txt'}


In [11]:
print(f"You have {len(document)} document.")

You have 1 document.


In [12]:
print(f"Your document has {len(document[0].page_content)} characters")

Your document has 27419 characters


**Split the document in small chunks**

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [14]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=400
)

In [15]:
document_chunks = text_splitter.split_documents(document)

In [16]:
print(type(document_chunks))
print(f"Now you have {len(document_chunks)} chunks.")

<class 'list'>
Now you have 12 chunks.


**Convert text chunks in numeric vectors (called "embeddings")**

In [17]:
from langchain.embeddings.openai import OpenAIEmbeddings

In [18]:
embeddings = OpenAIEmbeddings()

/Users/anshulvirmani/.pyenv/versions/3.11.4/envs/environmentname1/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


**Load the embeddings to a vector database**

In [21]:
from langchain.vectorstores import FAISS

In [22]:
#!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 2.8 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


*Careful: the next operation is expensive in OpenAI*

In [23]:
stored_embeddings = FAISS.from_documents(document_chunks, embeddings)

**Create a Retrieval Question & Answering Chain**

In [24]:
from langchain.chains import RetrievalQA

In [25]:
QA_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=stored_embeddings.as_retriever()
)

**Now we have a Question & Answering APP**

In [26]:
question = """
What is this article about? 
Describe it in less than 100 words.
"""

In [27]:
QA_chain.run(question)

/Users/anshulvirmani/.pyenv/versions/3.11.4/envs/environmentname1/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\nThis article is about how to have a successful startup. It discusses the importance of creating something people want and not worrying too much about a business model at first. It also talks about the possibility of failure and how to avoid it by not giving up and staying motivated. The author shares personal experiences and observations from working with startups. The main takeaway is that avoiding failure is the key to getting rich in a startup.'

In [28]:
question2 = """
And how does it explain how to create somethin people want?
"""

In [29]:
QA_chain.run(question2)

'\nThe essay suggests that being benevolent and genuinely trying to help people can lead to creating something people want. This is because when you are committed to helping others, you will keep working even when things seem difficult or your startup appears to be failing. This determination and dedication can lead to improving your product or service, making it more appealing to users. Additionally, being good and helping others can also make other people want to help you, which can provide valuable insights and assistance in creating something that people want.'

In [32]:
question3 = """
Does it have anything that a product manager can use?
"""

In [33]:
QA_chain.run(question3)

'\nYes, this passage contains advice for startups and entrepreneurs that a product manager could potentially use in their work. The overall message is to focus on creating something that users truly love, as this will improve morale, attract support, and ultimately lead to success. The passage also suggests that being benevolent and creating a product that genuinely helps people can be a successful strategy for a startup. Additionally, the passage touches on the importance of constantly iterating and not giving up, even when faced with challenges. These are all valuable lessons for a product manager to keep in mind when developing and managing a product.'

In [34]:
question4 = """
Can you create a 200 words long blog post using information presented in this text that could be helpful for a product manager?
"""

In [35]:
QA_chain.run(question4)

"\n\nProduct managers are often faced with the challenge of creating something that users truly love. However, this can be a daunting task, and it's easy to get demoralized when things don't seem to be going well. In these moments, it's important to remember the advice of successful entrepreneur Paul Buchheit: as long as your product has a core group of devoted users, you're on the right track. These users can provide valuable insights into what makes your product great, and can help guide your future decisions. Even if your product is not initially successful, don't give up. Instead, keep iterating and improving, and eventually you will find the winning formula. It's also important to stay focused on your startup and not get distracted by other projects or commitments. Distraction can be fatal to startups, and it's important to stay committed to your mission. Remember, failure is not the end as long as you keep trying. In fact, failure can often lead to success, as long as you learn f

In [36]:
question5 = """
How do I create a .env file on mac?
"""

In [38]:
QA_chain.run(question5)

'\nTo create a .env file on mac, follow these steps:\n\n1. Open Terminal and navigate to the directory where you want to create the .env file.\n\n2. Type "touch .env" and press enter. This will create a new, empty .env file in the current directory.\n\n3. Open the .env file in a text editor (such as TextEdit or Sublime Text) and add your environment variables in the format "VARIABLE=VALUE". For example: \n\nDB_HOST=localhost\nDB_USER=admin\nDB_PASSWORD=12345\n\n4. Save the file and close it.\n\n5. You can now use the .env file in your project to access these environment variables.'